<a href="https://colab.research.google.com/github/JonathanSum/Practice_Deep-Learning/blob/main/Building_the_model_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transform the data

In [1]:
%matplotlib inline

In [2]:
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [7]:
device

'cuda'

In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) 
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


In [9]:
pred_probab

tensor([[0.0969, 0.0969, 0.1018, 0.0969, 0.1086, 0.1016, 0.1051, 0.0986, 0.0969,
         0.0969]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [12]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [13]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear

In [14]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [15]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0995,  0.2572,  0.1182,  0.0042, -0.1727,  0.0698, -0.3668,  0.0033,
          0.3445,  0.4188,  0.4201,  0.1370,  0.4626,  0.1848, -0.0737,  0.1865,
         -0.1285, -0.1891,  0.1093, -0.2311],
        [-0.0333, -0.0231, -0.1633, -0.1824, -0.4598, -0.0675, -0.4594, -0.3915,
          0.3060,  0.4031,  0.5419, -0.4408,  0.4587,  0.2485,  0.2488, -0.0023,
          0.0042, -0.3388,  0.5323,  0.1544],
        [-0.1586, -0.0398,  0.1261, -0.2078, -0.3479,  0.2078, -0.6665, -0.1964,
          0.3376,  0.3207,  0.6493,  0.1710,  0.5272,  0.2005,  0.0210,  0.1435,
         -0.0905, -0.3942,  0.4907, -0.1351]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.2572, 0.1182, 0.0042, 0.0000, 0.0698, 0.0000, 0.0033, 0.3445,
         0.4188, 0.4201, 0.1370, 0.4626, 0.1848, 0.0000, 0.1865, 0.0000, 0.0000,
         0.1093, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3060,
         0.4031, 0.5419, 0.0000, 0.4587, 0.2485, 0.248

In [16]:
hidden1.shape

torch.Size([3, 20])

In [18]:
hidden1.size()

torch.Size([3, 20])

In [19]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [22]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [23]:
pred_probab

tensor([[0.0889, 0.0896, 0.0879, 0.0936, 0.0766, 0.1115, 0.0966, 0.1169, 0.1035,
         0.1348],
        [0.0787, 0.1018, 0.0946, 0.1193, 0.0944, 0.1041, 0.0810, 0.1142, 0.0986,
         0.1132],
        [0.0866, 0.1065, 0.0908, 0.0957, 0.0919, 0.1060, 0.0958, 0.1073, 0.0958,
         0.1236]], grad_fn=<SoftmaxBackward>)

In [24]:
pred_probab.argmax(-1)

tensor([9, 3, 9])

In [25]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0156, -0.0034, -0.0180,  ..., -0.0090, -0.0288,  0.0029],
        [ 0.0043, -0.0119, -0.0010,  ...,  0.0232,  0.0071,  0.0327]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0178, -0.0144], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0065,  0.0326, -0.0308,  ...,  0.0279, -0.0283, -0.0100],
        [-0.0012, -0.0018, -0.0434,  ...,  0.0099, -0.0326,  0.0435]],
       device='cuda